In [ ]:
%pip uninstall mordecai

In [184]:
import requests
from bs4 import BeautifulSoup as bs
import urllib.parse
import datetime
import re
import zipfile

import os
import zipfile
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import geopy.geocoders as gg
import geopandas as gpd
from geopandas.tools import geocode
from shapely.geometry import LineString, MultiPolygon

import folium
from folium import Choropleth, Circle, Marker, GeoJson
from folium.plugins import HeatMap, MarkerCluster

from IPython.display import display, Image, HTML, IFrame, FileLink, FileLinks
from IPython.core.interactiveshell import InteractiveShell
import ipywidgets as widgets

from sklearn import preprocessing
from sklearn.cluster import KMeans

import json
#InteractiveShell.ast_node_interactivity = "all"

%matplotlib inline


In [98]:
zf = zipfile.ZipFile('GEDEvent_v23_1.csv.zip') 
df = pd.read_csv(zf.open('GEDEvent_v23_1.csv'))

/tmp/ipykernel_10586/3301354906.py:2: DtypeWarning: Columns (47) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(zf.open('GEDEvent_v23_1.csv'))


In [100]:
select_columns = ['id', 'year', 'type_of_violence', 'conflict_name', 'dyad_name', 'side_a', 'side_b',
                  'where_coordinates', 'where_description', 'adm_1', 'adm_2', 'latitude', 'longitude', 'geom_wkt', 'country', 'region',
                  'date_start', 'date_end', 'deaths_a', 'deaths_b', 'deaths_civilians', 'best']

df_conflict = df[select_columns]
df_conflict = df_conflict.sort_values(['year', 'region', 'country'])

In [101]:
df_conflict.head()
df_conflict['date_start'] = pd.to_datetime(df_conflict['date_start'])
df_conflict['date_end'] = pd.to_datetime(df_conflict['date_end'])
df_conflict.info()


<class 'pandas.core.frame.DataFrame'>
Index: 316818 entries, 46025 to 316711
Data columns (total 22 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   id                 316818 non-null  int64         
 1   year               316818 non-null  int64         
 2   type_of_violence   316818 non-null  int64         
 3   conflict_name      316818 non-null  object        
 4   dyad_name          316818 non-null  object        
 5   side_a             316818 non-null  object        
 6   side_b             316818 non-null  object        
 7   where_coordinates  316818 non-null  object        
 8   where_description  310051 non-null  object        
 9   adm_1              298979 non-null  object        
 10  adm_2              250153 non-null  object        
 11  latitude           316818 non-null  float64       
 12  longitude          316818 non-null  float64       
 13  geom_wkt           316818 non-null  object   

In [105]:
eu_conflict = df_conflict[df_conflict.region == 'Europe']
eu_conflict = eu_conflict.reset_index(drop=True)

In [157]:
brit_events = eu_conflict[eu_conflict.country == 'United Kingdom']


In [161]:
brit_conflicts = brit_events["conflict_name"].unique()
trouble_conflicts = [brit_conflicts[0], brit_conflicts[1]]
trouble_events = brit_events[brit_events.conflict_name.isin(trouble_conflicts)]
trouble_events = trouble_events[trouble_events.year<2002]
#trouble_events

In [162]:
# ie_dths_yr = trouble_events.groupby('year')["best"].sum().rename("total_year").reset_index()
# ie_civs_yr = trouble_events.groupby('year')["deaths_civilians"].sum().rename("civilians_year").reset_index()

trouble_cities = trouble_events["where_description"].unique()
trouble_cities.sort()
trouble_cities


array(['Armagh town', 'Ballycastle town', 'Ballymoney town',
       'Belcoo village', 'Belfast Town (Ardoyne area)',
       'Belfast Town (New Lodge area of North Belfast)', 'Belfast town',
       'Bellaghy village', 'Belleek village', 'Bessbrook village',
       'Brookeborough village', 'Cappagh village', 'Castlederg village',
       'Castlerock village',
       'Castor Bay, near Morrows Point, Lough Neagh,', 'Coagh village',
       'Coalisland town', 'Comber town .', 'Cookstown town',
       'Crossgar village', 'Crossmaglen village', 'Cullyhanna village',
       'Deal town', 'Derry town', 'Donaghcloney village',
       'Donaghmore village (near Caste Caulfield)', 'Downpatrick town',
       'Drumquin village', 'Dungannon town', 'Fivemiletown village',
       'Forkhill village', 'Garvagh village', 'Glenanne village',
       'Greysteel village', 'Hamiltonsbawn village',
       'Jonesborough village', 'Keady village', 'Kilrea town',
       'Lichfield town', 'Lisbellaw  village', 'Lisburn

In [164]:
armagh_events = trouble_events[trouble_events['where_description'] == 'Armagh town']
armagh_events

,id,year,type_of_violence,conflict_name,dyad_name,side_a,side_b,where_coordinates,where_description,adm_1,...,longitude,geom_wkt,country,region,date_start,date_end,deaths_a,deaths_b,deaths_civilians,best
104,66300,1990,1,United Kingdom: Northern Ireland,Government of United Kingdom - PIRA,Government of United Kingdom,PIRA,Armagh town,Armagh town,Northern Ireland,...,-6.66667,POINT (-6.66667 54.35),United Kingdom,Europe,1990-01-22,1990-01-22,1,0,0,1
108,66324,1990,1,United Kingdom: Northern Ireland,Government of United Kingdom - PIRA,Government of United Kingdom,PIRA,Armagh town,Armagh town,Northern Ireland,...,-6.66667,POINT (-6.66667 54.35),United Kingdom,Europe,1990-03-28,1990-03-28,1,0,0,1
8201,67358,1993,1,United Kingdom: Northern Ireland,Government of United Kingdom - PIRA,Government of United Kingdom,PIRA,Armagh town,Armagh town,Northern Ireland,...,-6.66667,POINT (-6.66667 54.35),United Kingdom,Europe,1993-02-09,1993-02-09,1,0,0,1
10199,66530,1994,1,United Kingdom: Northern Ireland,Government of United Kingdom - PIRA,Government of United Kingdom,PIRA,Armagh town,Armagh town,Northern Ireland,...,-6.66667,POINT (-6.66667 54.35),United Kingdom,Europe,1994-05-21,1994-05-21,1,0,0,1


In [165]:
belfast = [ct for ct in trouble_cities if ("belfast" in ct.lower())]
print(belfast)
derry = [ct for ct in trouble_cities if ("derry" in ct.lower())]
print(derry)

['Belfast Town (Ardoyne area)', 'Belfast Town (New Lodge area of North Belfast)', 'Belfast town', 'near Belfast town (Glengormley)']
['Derry town', 'Londonderry', 'Londonderry town']


In [166]:
def get_victim_info(datum):

    index = "https://cain.ulster.ac.uk/sutton/chron/index.html"
    url_parse_results = urllib.parse.urlparse(index)
    url_parse_results
    base = "://".join([url_parse_results.scheme, url_parse_results.netloc])
    r = requests.get(index)
    soup = bs(r.content)    

    
    year =  datum.year
    month = datum.month
    day = datum.day

    date = datetime.date(year, month, day)
    str_date = date.strftime("%d %B %Y")

    link = soup.find_all('a', href=True, string = year)
    if len(link) > 1:
        raise ReferenceError

    targetURL = "".join([base, link[0]["href"]])
    req = requests.get(targetURL)
    db_soup = bs(req.content)

    ppl = db_soup.find_all(True, string = re.compile(str_date))

    persons ={}

    for p in ppl:
        d = [text for text in p.parent.stripped_strings]
        im = "".join([base, p.parent.previous_sibling.contents[0]["src"]])


        name = d[1]
        age = d[2].split(")")[0][1:]
        religion = d[2].split(")")[-1].strip()
        status = d[4].strip()[:-1]
        killer = d[6].strip()
        location = d[7].strip()

        info_dict = {
                #"name": name,
                "age": age,
                "religion": religion,
                "status": status,
                "killed_by": killer,
                "location": location,
                "image": im
        }
        persons.update({name: info_dict})

    return(persons)


In [171]:
amargh_deaths = {}

for ind in armagh_events.index:
    date_start = armagh_events["date_start"][ind]
    date_end = armagh_events["date_end"][ind]
    duration = date_end- date_start
    if duration:
        raise ReferenceError
    
    victims = get_victim_info(date_start)
    amargh_deaths.update(victims)

print(amargh_deaths)

22 January 1990
{'Monteith, Derek': {'age': '35', 'religion': 'Protestant', 'status': 'Royal Ulster Constabulary (RUC)', 'killed_by': 'Irish Republican Army (IRA)', 'location': 'Off duty. Shot at his home, Kilburn Park, Armagh.', 'image': 'https://cain.ulster.ac.uk/victims/humanface/photos/Monteith_Derek_220190.jpg'}}
28 March 1990
{'Starrett, George': {'age': '58', 'religion': 'Protestant', 'status': 'Royal Ulster Constabulary (RUC)', 'killed_by': 'Irish Republican Army (IRA)', 'location': 'Off duty. Shot at his home, Newry Road, Armagh.', 'image': 'https://cain.ulster.ac.uk/victims/humanface/photos/Starrett_George_280390.jpg'}}
09 February 1993
{'Beswick, Michael': {'age': '21', 'religion': 'nfNI', 'status': 'British Army (BA)', 'killed_by': 'Irish Republican Army (IRA)', 'location': 'Killed by remote controlled bomb hidden in wall, detonated when British Army (BA) foot patrol passed, Cathedral Road, Armagh.', 'image': 'https://cain.ulster.ac.uk/victims/humanface/photos/Beswick_Micha

In [187]:
index = "https://cain.ulster.ac.uk/sutton/chron/index.html"
url_parse_results = urllib.parse.urlparse(index)
url_parse_results
base = "://".join([url_parse_results.scheme, url_parse_results.netloc])
r = requests.get(index)
soup = bs(r.content) 

def get_victim_info(datum):
  
    info_dict = {}
    
    year =  datum.year
    month = datum.month
    day = datum.day

    date = datetime.date(year, month, day)
    str_date = date.strftime("%d %B %Y")

    link = soup.find_all('a', href=True, string = year)
    if len(link) > 1:
        raise ReferenceError

    targetURL = "".join([base, link[0]["href"]])
    req = requests.get(targetURL)
    db_soup = bs(req.content)

    ppl = db_soup.find_all(True, string = re.compile(str_date))

    #persons ={}

    for p in ppl:
        d = [text for text in p.parent.stripped_strings]
        im = "".join([base, p.parent.previous_sibling.contents[0]["src"]])


        name = d[1]
        age = d[2].split(")")[0][1:]
        religion = d[2].split(")")[-1].strip()
        status = d[4].strip()[:-1]
        killer = d[6].strip()
        location = d[7].strip()

        info_dict = {
                "name": name,
                "age": age,
                "religion": religion,
                "status": status,
                "killed_by": killer,
                "location": location,
                "image": im
        }
        #persons.update({id: info_dict})

    return(info_dict)


In [188]:
victims = {}

for ind in trouble_events.index:
    date_start = trouble_events["date_start"][ind]
    date_end = trouble_events["date_end"][ind]
    delta = datetime.timedelta(days=1)
    id = int(trouble_events["id"][ind])
    while (date_start <= date_end):
        vic = get_victim_info(date_start)
        if id in victims:
            victims[id].append(vic)
        else:
            victims[id] = [vic]     
        date_start += delta
    


print(victims)

ConnectTimeout: HTTPSConnectionPool(host='cain.ulster.ac.uk', port=443): Max retries exceeded with url: /sutton/chron/1990.html (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7fcb75c14dc0>, 'Connection to cain.ulster.ac.uk timed out. (connect timeout=None)'))

In [189]:
with open("trouble_victims", "w", encoding='utf-8') as file:
    json.dump(victims, file, ensure_ascii=False, indent=4)
victims

{66877: [{'name': 'Montgomery, Stephen',
   'age': '26',
   'religion': 'Protestant',
   'status': 'Royal Ulster Constabulary (RUC)',
   'killed_by': 'Irish Republican Army (IRA)',
   'location': 'Killed in grenade attack on stationary Royal Ulster Constabulary (RUC) mobile patrol, Sion Mills, County Tyrone.',
   'image': 'https://cain.ulster.ac.uk/victims/humanface/photos/Montgomery_Stephen_280189.jpg'}],
 66878: [{'name': 'Peacock, Nicholas',
   'age': '20',
   'religion': 'nfNI',
   'status': 'British Army (BA)',
   'killed_by': 'Irish Republican Army (IRA)',
   'location': 'Killed by remote controlled bomb, hidden in drainpipe, while on British Army (BA) foot patrol, Rockmore Street, Falls, Belfast.',
   'image': 'https://cain.ulster.ac.uk/victims/humanface/photos/nophoto.jpg'}],
 66885: [{'name': 'Connolly, James',
   'age': '20',
   'religion': 'Catholic',
   'status': 'Irish Republican Army (IRA)',
   'killed_by': 'Irish Republican Army (IRA)',
   'location': "Died in premature 

In [172]:
len(amargh_deaths)

5

In [125]:
index = "https://cain.ulster.ac.uk/sutton/chron/index.html"
url_parse_results = urllib.parse.urlparse(index)
url_parse_results
base = "://".join([url_parse_results.scheme, url_parse_results.netloc])
r = requests.get(index)
soup = bs(r.content)

In [92]:
r = requests.get(targetURL)
db_soup = bs(r.content)


In [93]:
ppl = db_soup.find_all(True, string = re.compile(str_date))

persons ={}

for p in ppl:
    d = [text for text in p.parent.stripped_strings]
    im = "".join([base, p.parent.previous_sibling.contents[0]["src"]])


    name = d[1]
    age = d[2].split(")")[0][1:]
    religion = d[2].split(")")[-1].strip()
    status = d[4].strip()[:-1]
    killer = d[6].strip()
    location = d[7].strip()

    info_dict = {
            #"name": name,
            "age": age,
            "religion": religion,
            "status": status,
            "killed_by": killer,
            "location": location,
            "image": im
    }
    persons.update({name: info_dict})

persons

{"O'Hagan, Martin": {'age': '51',
  'religion': 'Catholic',
  'status': 'Civilian (Civ)',
  'killed_by': 'Red Hand Defenders (RHD)',
  'location': 'Journalist.  Shot while walking near to his home, Westfield Gardens, off Tandragee Road, Lurgan, County Armagh.',
  'image': 'https://cain.ulster.ac.uk/victims/humanface/photos/OHagan_Martin_280901.jpg'}}